In [6]:
import matplotlib.pyplot as plt
import numpy as np
import time

In [7]:
### define initial values
n_years = 100
dt = 1/12
n_factors = 2
n_trials = 10000
rho = -0.739
x0 = 0.0228
y0 = 0.0809
alpha_ls = np.matrix([1.0682,0.0469]).reshape(2,1)
mu_ls = np.matrix([0.0546,0.0778]).reshape(2,1)
sigma_ls = np.matrix([ 0.0412,0.0287]).reshape(2,1)

In [8]:
### Correlated Weiner Process generation
start = time.time()
X = np.random.standard_normal((n_factors,int(n_years*1/dt),n_trials))
x = np.matrix([x0,y0])

start = time.time()
corre_mat = np.matrix([1,rho,rho,1]).reshape(2,2)
L = np.linalg.cholesky(corre_mat)
correlateWeinerProcess = np.sqrt(dt)*(np.einsum('ij,ikl -> ikl',L,X))

end1 = time.time()

### just reshaping initial x vector and static parameters for tensor form
x = np.zeros((2,1,n_trials))
x[0,:,:] = x0
x[1,:,:] = y0

mu = np.zeros((2,1,n_trials))
mu[0,:,:] = mu_ls[0]
mu[1,:,:] = mu_ls[1]

alpha = np.zeros((2,1,n_trials))
alpha[0,:,:] = alpha_ls[0]
alpha[1,:,:] = alpha_ls[1]

sigma = np.zeros((2,1,n_trials))
sigma[0,:,:] = sigma_ls[0]
alpha[1,:,:] = sigma_ls[1]

### time step loop
final_x = [x]
for i in range(int(n_years/dt)-1):
    dzt = correlateWeinerProcess[:,i,:].reshape(2,1,n_trials)
    xt = final_x[-1]
    drift = np.einsum(',ijk->ijk',dt,np.einsum('ijk,ijk -> ijk', alpha, (mu-xt)))
    diffusion = np.einsum('ijk,ijk->ijk',np.einsum('ijk,ijk->ijk',sigma,np.sqrt(xt)),dzt)
    final_x.append(xt+drift+diffusion)

### final calculations
final_x = np.array(final_x)#.reshape(2,int(n_years/dt),n_trials)
xt1 = np.sum(final_x,axis=1)
phi = 0.045 - np.sum(x,axis=0)
n = phi + xt1

end = time.time()

In [12]:
print(f"Weiner Process Time : {end1-start}")
print(f"Time Step Iteration Process Time : {end-end1}")

Weiner Process Time : 0.3673689365386963
Time Step Iteration Process Time : 0.36202287673950195
